# Usage reference guide for haystack-reverse

this is an example of every haystack-reverse commands.

The zeus.vmem.856.dump is there https://dl.dropboxusercontent.com/u/10222931/HAYSTACK/zeus.vmem.856.dump.tgz

It was extracted from pid 856 from the zeus.img image from http://malwarecookbook.googlecode.com/svn-history/r26/trunk/17/1/zeus.vmem.zip

In [11]:
!haystack-reverse --help

usage: haystack-reverse [-h] [--debug] dumpname {analyze,types,strings,show,hex,parents,clean} ...

Several tools to reverse engineer records from heap.

positional arguments:
  dumpname              Memory dump filename
  {analyze,types,strings,show,hex,parents,clean}
                        sub-command help
    analyze             Step 1. Run heuristics to reverse allocations record types
    types               Show reversed record types
    strings             Show fields that are assumed to be strings types
    show                Show one record instance
    hex                 Show the bytes for one record instance
    parents             List the predecessors pointing to the record at this address
    clean               Clean the memory dump from cached info.

optional arguments:
  -h, --help            show this help message and exit
  --debug               Debug mode on.


First we need to generate the analysis for the process memory dump.

In [10]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump analyze

INFO:reverse.api:Reversing Fields
INFO:model:[+] <FieldReverser>: START
INFO:model:[+] <FieldReverser>: START on heap 0x90000
INFO:model:[+] <FieldReverser>: START on heap 0x190000
INFO:model:[+] <FieldReverser>: START on heap 0x1a0000
INFO:model:[+] <FieldReverser>: START on heap 0x7f6f0000
INFO:model:[+] <FieldReverser>: START on heap 0x3b0000
INFO:model:[+] <FieldReverser>: START on heap 0x350000
INFO:model:[+] <FieldReverser>: START on heap 0x410000
INFO:model:[+] <FieldReverser>: START on heap 0xc30000
INFO:model:[+] <FieldReverser>: START on heap 0xd60000
INFO:model:[+] <FieldReverser>: START on heap 0xe20000
INFO:model:[+] <FieldReverser>: START on heap 0xe80000
INFO:reverse.api:Fixing Text Fields
INFO:model:[+] <TextFieldCorrection>: START
INFO:model:[+] <TextFieldCorrection>: START on heap 0x90000
INFO:model:[+] <TextFieldCorrection>: START on heap 0x190000
INFO:model:[+] <TextFieldCorrection>: START on heap 0x1a0000
INFO:model:[+] <TextFieldCorrection>: START on heap 0x7f6f00

Then we can start to use some of the cli

In [5]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump strings| grep -a http

0xc64e8,0x40 bytes,u'http://193.104.41.75/cbd/75.bro\x00'
0xc32d98,0x20 bytes,'http://193.104.41.75/cbd/75.bro '
0xc329f8,0x40 bytes,'Ahttps://onlineeast#.bankofamerica.com/cgi-bin/ias/*/GotoWelcome'


Mhh interesting string... I wonder what memory chunk was that allocated in.

In [6]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump show 0xc64e8

INFO:haystack-reverse:[+] Found record at: @c64e8

class struct_c64e8(ctypes.Structure):  # rlevel:50 SIG:T64 size:64
  _fields_ = [ 
	( 'utf16_0' , ctypes.c_char*64 ), #  # ctypes.c_char: u'http://193.104.41.75/cbd/75.bro\x00'
 ]




Ah, that make sense.. It's a classic utf16 string. The whole allocated memory chunk is being used for a string.

Lets look at the bytes behind the scene.

In [11]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump hex 0xc64e8

INFO:haystack-reverse:[+] Found record at: @c64e8
'h\x00t\x00t\x00p\x00:\x00/\x00/\x001\x009\x003\x00.\x001\x000\x004\x00.\x004\x001\x00.\x007\x005\x00/\x00c\x00b\x00d\x00/\x007\x005\x00.\x00b\x00r\x00o\x00\x00\x00'


I wonder if this record was referenced in some other record...
Maybe we can find a parent record that points to this string...

In [2]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump parents 0xc64e8

INFO:haystack-reverse:[+] Showing predecessors of record at: 0xc64e8
# [+] No parents records found.


Tough luck... What about the others ?

In [1]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump parents 0xc32d98

INFO:haystack-reverse:[+] Showing predecessors of record at: 0xc32d98
# [+] No parents records found.


In [10]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump parents 0xc329f8

INFO:haystack-reverse:[+] Showing predecessors of record at: 0xc329f8
#0xc31e90

class struct_c31e90(ctypes.Structure):  # rlevel:50 SIG:P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4a8 size:336
  _fields_ = [ 
	( 'ptr_string_c32000_0' , ctypes.c_void_p ), # @ 0x00c32000 # [vol_mapping_058]
	( 'ptr_string_c32020_4' , ctypes.c_void_p ), # @ 0x00c32020 # [vol_mapping_058]
	( 'ptr_string_c32040_8' , ctypes.c_void_p ), # @ 0x00c32040 # [vol_mapping_058]
	( 'ptr_string_c32060_12' , ctypes.c_void_p ), # @ 0x00c32060 # [vol_mapping_058]
	( 'ptr_string_c320a8_16' , ctypes.c_void_p ), # @ 0x00c320a8 # [vol_mapping_058]
	( 'ptr_string_c320d0_20' , ctypes.c_void_p ), # @ 0x00c320d0 # [vol_mapping_058]
	( 'ptr_string_c320f8_24' , ctypes.c_void_p ), # @ 0x00c320f8 # [vol_mapping_058]
	( 'ptr_string_c32120_28' , ctypes.c_void_p ), # @ 0x00c32120 # [vol_mapping_058]
	( 'ptr_string_c32

That looks interesting. A record made of 82x 4-bytes pointers and some trailings zeroes/padding.

Let's see if we can check that out with haystack CLI.


In [15]:
!cat ../test/structures/zeus/records.py

import ctypes


class array_of_pointers(ctypes.Structure):
    _fields_ = [('array', ctypes.CString*82)]


class array_of_wcharp(ctypes.Structure):
    _fields_ = [('array', ctypes.CWString*82)]


So, due to a little monkey patching, there is a CString ctypes types available in the haystack ctypes module.

In [17]:
!haystack --dumpname ../test/dumps/vol/zeus.vmem.856.dump --string refresh test.structures.zeus.records.array_of_pointers 0xc31e90

[# --------------- 0x1 
{ # <array_of_pointers at 0xc31e90>
"array": [
	"l" , # (CString)
	"u" , # (CString)
	"l" , # (CString)
	"s" , # (CString)
	"w" , # (CString)
	"s" , # (CString)
	"e" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"s" , # (CString)
	"U" , # (CString)
	"s" , # (CString)
	"s" , # (CString)
	"u" , # (CString)
	"software" , # (CString)
	"system" , # (CString)
	"%" , # (CString)
	"%" , # (CString)
	"n" , # (CString)
	"o" , # (CString)
	"z" , # (CString)
	"*%u.%u.%u.%u*" , # (CString)
	"winsta0" , # (CString)
	"default" , # (CString)
	"gdiplus.dll" , # (CString)
	"ole32.dll" , # (CString)
	"gdi32.dll" , # (CString)
	"DISPLAY" , # (CString)
	"GdiplusStartup" , # (CString)
	"GdiplusShutdown" , # (CString)
	"GdipCreateBitmapFromHBITMAP" , # (CString)
	"GdipDisposeImage" , # (CString)
	"GdipGetImageEncodersSize" , # (CString)
	"GdipGetImageEncoders" , # (CString)
	"GdipSaveImageToStream" , # (CString)
	"CreateS

Oh, that is pretty good...
but it seems the first few strings are not quite right..

In [14]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump hex 0x00c32000
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump show 0x00c32000

INFO:haystack-reverse:[+] Found record at: @c32000
'l\x00o\x00w\x00s\x00e\x00c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
INFO:haystack-reverse:[+] Found record at: @c32000

class string_c32000(ctypes.Structure):  # rlevel:50 SIG:T12a12 size:24
  _fields_ = [ 
	( 'utf16_0' , ctypes.c_char*12 ), #  # ctypes.c_char: u'lowsec'
	( 'zerroes_12' , ctypes.c_ubyte*12 ), # # array
 ]




Mhh, it seems the first few strings are utf16 strings. Lets try with a Wide char string type.


In [18]:
!haystack --dumpname ../test/dumps/vol/zeus.vmem.856.dump --string refresh test.structures.zeus.records.array_of_wcharp 0xc31e90  

[# --------------- 0x1 
{ # <array_of_wcharp at 0xc31e90>
"array": [
	"l o w s e c" , # (CWString)
	"u s e r . d s" , # (CWString)
	"l o c a l . d s" , # (CWString)
	"s d r a 6 4 . e x e" , # (CWString)
	"w i n l o g o n . e x e" , # (CWString)
	"s v c h o s t . e x e" , # (CWString)
	"e x p l o r e r . e x e" , # (CWString)
	"_ A V I R A _ 2 1 1 0" , # (CWString)
	"_ A V I R A _ 2 1 0 1" , # (CWString)
	"_ A V I R A _ 2 1 0 8" , # (CWString)
	"_ A V I R A _ 2 1 0 9" , # (CWString)
	"_ A V I R A _ 2 1 0 9 9" , # (CWString)
	"s o f t w a r e \ m i c r o s o f t \ w i n d o w s   n t \ c u r r e n t v e r s i o n \ n e t w o r k" , # (CWString)
	"U I D" , # (CWString)
	"s o f t w a r e \ m i c r o s o f t \ w i n d o w s   n t \ c u r r e n t v e r s i o n \ w i n l o g o n" , # (CWString)
	"s o f t w a r e \ m i c r o s o f t \ w i n d o w s \ c u r r e n t v e r s i o n \ r u n" , # (CWString)
	"u s e r i n i t" , # (CWString)
	"software" , # (CWString)
	"system" , # (CWString)
	"% 0 8

Well, that more or less works.


Ultimately, you might want to clean the cache ( REMOVES ALL ANALYSIS FILES )

In [20]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump clean

INFO:haystack-reverse:Removing cache folder /home/other/Compil/python-haystack/test/dumps/vol/zeus.vmem.856.dump/cache
